# Ejercicio prueba
Consta en hacer simples tareas de pandas pero ahora con la sintaxis de polars.
Los csv de emisiones-2016.csv, emisiones-2017.csv, emisiones-2018.csv y emisiones-2019.csv, contienen datos sobre las emisiones contaminates en la ciudad de Madrid en los años 2016, 2017, 2018 y 2019 respectivamente. Escribir un programa con los siguientes requisitos:

*Es importante que el código ea breve, ya que cada ejercicio puede hacerse en una sola instrucción* :OO

### Generar un DataFrame con los datos de los cuatro ficheros.
Hint: leer los datos y concatenarlos verticalmente --> **pl.concat**

In [2]:
import polars as pl
anio2016 = pl.read_csv("emisiones-2016.csv", separator = ";")
anio2017 = pl.read_csv("emisiones-2017.csv", separator = ";")
anio2018 = pl.read_csv("emisiones-2018.csv", separator = ";")
anio2019 = pl.read_csv("emisiones-2019.csv", separator = ";")
df = pl.concat([anio2016, anio2017, anio2018, anio2019])

## Código aquí


### Filtrar las columnas del DataFrame para quedarse únicamente con las columnas de ESTACION, MAGNITUD, AÑO, MES y las correspondientes a todos los días D01, D02, ... 
Hint: **df.select(...)** 

In [3]:
## Código aquí

# VAMOS A SELECCIONAR HASTA D31
df = df.select(["ESTACION", "MAGNITUD", "ANO", "MES", "D01", "D02", "D03", "D04", "D05", "D06", "D07", "D08", "D09", "D10", "D11", "D12", "D13", "D14", "D15", "D16", "D17", "D18", "D19", "D20", "D21", "D22", "D23", "D24", "D25", "D26", "D27", "D28", "D29", "D30", "D31"])


### Reestructurar el DataFrame para que los valores de los contaminantes de las columnas de los días aparezcan en una única columna.


In [4]:
## Código aquí
df = df.melt(id_vars=["ESTACION", "MAGNITUD", "ANO", "MES"],variable_name="DIA", value_name="VAlOR")
print(df)

shape: (225_246, 6)
┌──────────┬──────────┬──────┬─────┬─────┬───────┐
│ ESTACION ┆ MAGNITUD ┆ ANO  ┆ MES ┆ DIA ┆ VAlOR │
│ ---      ┆ ---      ┆ ---  ┆ --- ┆ --- ┆ ---   │
│ i64      ┆ i64      ┆ i64  ┆ i64 ┆ str ┆ f64   │
╞══════════╪══════════╪══════╪═════╪═════╪═══════╡
│ 4        ┆ 1        ┆ 2016 ┆ 1   ┆ D01 ┆ 8.0   │
│ 4        ┆ 1        ┆ 2016 ┆ 2   ┆ D01 ┆ 12.0  │
│ 4        ┆ 1        ┆ 2016 ┆ 3   ┆ D01 ┆ 11.0  │
│ 4        ┆ 1        ┆ 2016 ┆ 4   ┆ D01 ┆ 8.0   │
│ …        ┆ …        ┆ …    ┆ …   ┆ …   ┆ …     │
│ 60       ┆ 14       ┆ 2019 ┆ 9   ┆ D31 ┆ 0.0   │
│ 60       ┆ 14       ┆ 2019 ┆ 10  ┆ D31 ┆ 47.0  │
│ 60       ┆ 14       ┆ 2019 ┆ 11  ┆ D31 ┆ 0.0   │
│ 60       ┆ 14       ┆ 2019 ┆ 12  ┆ D31 ┆ 4.0   │
└──────────┴──────────┴──────┴─────┴─────┴───────┘


### Crear una nueva columna llamada FECHA con las fechas a partir del año, mes y día
Hint: **pl.concat_str([
            pl.col(...),
            ...
        ])** y **.str.strptime(pl.Datetime,strict=False)** 

In [10]:

df = df.with_columns([pl.col("DIA").str.replace(r"D", "")])

def formatear(x):
    return str(x)+"/"

df = df.with_columns(df.select([
    pl.concat_str([pl.col("ANO").apply(formatear), pl.col("MES").apply(formatear), pl.col("DIA")]).alias("FECHA").str.strptime(pl.Datetime, strict = False)
]))
print(df)

shape: (225_246, 7)
┌──────────┬──────────┬──────┬─────┬─────┬───────┬─────────────────────┐
│ ESTACION ┆ MAGNITUD ┆ ANO  ┆ MES ┆ DIA ┆ VAlOR ┆ FECHA               │
│ ---      ┆ ---      ┆ ---  ┆ --- ┆ --- ┆ ---   ┆ ---                 │
│ i64      ┆ i64      ┆ i64  ┆ i64 ┆ str ┆ f64   ┆ datetime[μs]        │
╞══════════╪══════════╪══════╪═════╪═════╪═══════╪═════════════════════╡
│ 4        ┆ 1        ┆ 2016 ┆ 1   ┆ 01  ┆ 8.0   ┆ 2016-01-01 00:00:00 │
│ 4        ┆ 1        ┆ 2016 ┆ 2   ┆ 01  ┆ 12.0  ┆ 2016-02-01 00:00:00 │
│ 4        ┆ 1        ┆ 2016 ┆ 3   ┆ 01  ┆ 11.0  ┆ 2016-03-01 00:00:00 │
│ 4        ┆ 1        ┆ 2016 ┆ 4   ┆ 01  ┆ 8.0   ┆ 2016-04-01 00:00:00 │
│ …        ┆ …        ┆ …    ┆ …   ┆ …   ┆ …     ┆ …                   │
│ 60       ┆ 14       ┆ 2019 ┆ 9   ┆ 31  ┆ 0.0   ┆ null                │
│ 60       ┆ 14       ┆ 2019 ┆ 10  ┆ 31  ┆ 47.0  ┆ 2019-10-31 00:00:00 │
│ 60       ┆ 14       ┆ 2019 ┆ 11  ┆ 31  ┆ 0.0   ┆ null                │
│ 60       ┆ 14       ┆ 2019 ┆ 

### Eliminar las filas con fechas no válidas

In [11]:
## Código aquí
df = df.drop_nulls()

### Ordenar el el dataframe por estación, magnitud y fecha
Hint: **.sort(..)**

In [12]:
## Código aquí
df = df.sort(by = ["ESTACION", "MAGNITUD", "FECHA"])

### Mostrar las estaciones y los contaminantes disponibles


In [13]:
## Código aquí
print(df.select([pl.col("ESTACION").n_unique().alias("Estaciones unicas")]))
print(df.select([pl.col("MAGNITUD").n_unique().alias("Contaminantes unicos")]))


shape: (1, 1)
┌───────────────────┐
│ Estaciones unicas │
│ ---               │
│ u32               │
╞═══════════════════╡
│ 24                │
└───────────────────┘
shape: (1, 1)
┌──────────────────────┐
│ Contaminantes unicos │
│ ---                  │
│ u32                  │
╞══════════════════════╡
│ 14                   │
└──────────────────────┘


## Tiempo de ejecución
Realiza una comparación de tiempo de ejecución utilizando pandas y polars. Lee todos los csv de https://github.com/pola-rs/polars/tree/main/examples/datasets, concatena verticalmente todos los csv y realiza un groupby a 'category' con cualquier función de agregación. De ahí compara el tiempo realizando 10000 iteraciones con la libreria `timeit` de python.

In [16]:
## Código aquí
archivos = ["foods1.csv", "foods2.csv", "foods3.csv", "foods4.csv", "foods5.csv"]
df = pl.concat([pl.read_csv("https://github.com/pola-rs/polars/tree/main/examples/datasets/{}".format(archivo)) for archivo in archivos])

import pandas as pd
import timeit
dfpandas = pd.concat(["https://github.com/pola-rs/polars/tree/main/examples/datasets/{}".format(archivo) for archivo in archivos])

polars_time = timeit.timeit(lambda: df.select("calories").groupby("category").sum(), number = 1000)
pandas_time = timeit.timeit(lambda: dfpandas.groupby("category")["calories"].sum(), number = 1000)


print("polars_time = {}, pandas_time = {}".format(polars_time, pandas_time))
